<a href="https://colab.research.google.com/github/julesy-thedev/CS482-Toxic-Tweets/blob/milestone-3/toxic_comments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install datasets transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 71.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.2/224.2 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 52.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 61.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.3/269.3 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 16.5 MB/s eta 0:00:00


In [ ]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf

from google.colab import drive
from zipfile import ZipFile
from datasets import load_dataset
from huggingface_hub import notebook_login
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers.schedules import PolynomialDecay
from tensorflow.keras.optimizers import Adam
from transformers import TFAutoModelForSequenceClassification, TFAutoModelForSequenceClassification, DataCollatorWithPadding, AutoTokenizer

batch_size = 8
num_epochs = 3

In [ ]:
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
def unzip(zip_file_loc):
  #Tutorial provided by https://www.geeksforgeeks.org/unzipping-files-in-python/
  with ZipFile(zip_file_loc, 'r') as zipFile:
    zipFile.extractall()
  zipFile.close()

In [ ]:
#Tutorial to setup Kaggle API was provided by https://medium.com/unpackai/how-to-use-kaggle-datasets-in-google-colab-f9b2e4b5767c
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Colab Notebooks/"
path = "/content/gdrive/My Drive/Colab Notebooks/Project/"
dataset_name = "jigsaw-toxic-comment-classification-challenge"

%cd "/content/gdrive/My Drive/Colab Notebooks/Project/"

!kaggle competitions download -c $dataset_name

unzip(path + dataset_name + ".zip")
unzip("test.csv.zip")
unzip("test_labels.csv.zip")
unzip("train.csv.zip")

!rm *.zip

/content/gdrive/My Drive/Colab Notebooks/Project
 95% 50.0M/52.6M [00:00<00:00, 85.7MB/s]
100% 52.6M/52.6M [00:00<00:00, 81.5MB/s]


In [ ]:
raw_train_dataset = load_dataset("csv", data_files="train.csv")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-06198ca3595e5ed3/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
raw_train = raw_train_dataset["train"]

test = raw_train.train_test_split(test_size=0.2, seed=42)

raw_train_dataset["train"] = test["train"]
raw_train_dataset["test"] = test["test"]

In [ ]:
labels = [label for label in raw_train_dataset["train"].features.keys() if label not in ["id", "comment_text"]]

id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize_function(batched_dataset):
  tokenized_batch_elements = tokenizer(batched_dataset["comment_text"], truncation=True)

  num_labels = len(labels)

  labels_2d = np.zeros((len(batched_dataset["comment_text"]), num_labels))

  colums_2_labels = {x: batched_dataset[x] for x in batched_dataset.keys() if x in labels}

  for i, label in enumerate(labels):
    labels_2d[:, i] = colums_2_labels[label]

  tokenized_batch_elements["labels"] = labels_2d.tolist()

  return tokenized_batch_elements

tokenized_dataset = raw_train_dataset.map(tokenize_function, remove_columns=raw_train_dataset["train"].column_names, batched=True)

Map:   0%|          | 0/127656 [00:00<?, ? examples/s]

Map:   0%|          | 0/25532 [00:00<?, ? examples/s]

Map:   0%|          | 0/6383 [00:00<?, ? examples/s]

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

tf_train_dataset = tokenized_dataset["train"].to_tf_dataset(
  columns = ["attention_mask", "input_ids"],
  label_cols = ["labels"],
  shuffle = True,
  collate_fn = data_collator,
  batch_size = batch_size,
)

tf_validation_dataset = tokenized_dataset["test"].to_tf_dataset(
  columns = ["attention_mask", "input_ids"],
  label_cols = ["labels"],
  shuffle = False,
  collate_fn = data_collator,
  batch_size = batch_size,
)

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
num_train_steps = len(tf_train_dataset) * num_epochs

lr_scheduler = PolynomialDecay(initial_learning_rate = 5e-5, 
                               end_learning_rate = 0.0, 
                               decay_steps = num_train_steps)

opt = Adam(learning_rate = lr_scheduler)

In [ ]:
model = TFAutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", problem_type="multi_label_classification", num_labels=len(labels), id2label=id2label, label2id=label2id)

model.compile(optimizer = opt,
              loss = BinaryCrossentropy(from_logits=True),
              metrics = ["accuracy"]
              )

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['activation_13', 'vocab_layer_norm', 'vocab_projector', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier', 'pre_classifier', 'dropout_19']
You should probably TRAIN this model on a down-stream task to be able to use i

In [ ]:
model.fit(tf_train_dataset, validation_data = tf_validation_dataset, epochs = num_epochs)

Epoch 1/3
15957/15957 [==============================] - 4421s 275ms/step - loss: 0.0488 - accuracy: 0.9874 - val_loss: 0.0398 - val_accuracy: 0.9940
Epoch 2/3
15957/15957 [==============================] - 4256s 267ms/step - loss: 0.0352 - accuracy: 0.9926 - val_loss: 0.0389 - val_accuracy: 0.9933
Epoch 3/3
15957/15957 [==============================] - 4204s 263ms/step - loss: 0.0256 - accuracy: 0.9935 - val_loss: 0.0414 - val_accuracy: 0.9922


In [ ]:
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
model.push_to_hub("toxic-comments-distilbert")

tf_model.h5:   0%|          | 0.00/268M [00:00<?, ?B/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
tokenizer.push_to_hub("toxic-comments-distilbert")

CommitInfo(commit_url='https://huggingface.co/julesy/toxic-comments-distilbert/commit/99c1b90f77d2bc66aeba45d78724c324c26a295c', commit_message='Upload tokenizer', commit_description='', oid='99c1b90f77d2bc66aeba45d78724c324c26a295c', pr_url=None, pr_revision=None, pr_num=None)